# Prediction

# Preprocesamiento

### Build images

The images for the prediction process are generated in the same way but in this case, the masks are not going to be created.
In the prediction process, the size and the step-size must be the same to do not generate an overlap in the predicted results. However, there is no need that the images have the same size as the training images.

In [ ]:
!satproc_extract_chips     \
    /home/ro/incendios-forestales/data/img/dif_nbr.tif \
    -o /home/ro/incendios-forestales/data/data_predict/160_80/ \
    --size 800 \    
    --step-size 800 \   
    --rescale \
    --rescale-mode percentiles \
    --upper-cut 98 --lower-cut 2atproc_extract_chips \


The **--aoi** option is useful to reduce the predicction area to some shapefile.

# Predict 

In [ ]:
from fire.unet.predict import PredictConfig, predict, plot_data_results
import os

In [ ]:
predict_config = PredictConfig(
                                    images_path=os.path.join('./data/data_predict/','800_800'),
                                    results_path=os.path.join('./data/data_result/','800_800'),
                                    batch_size=16,
                                    model_path=os.path.join('./data/weights/', 'UNet_fire_1Dim_160_80_spe150.h5'),  #  ruta al modelo (.h5)
                                    height=160,
                                    width=160,
                                    n_channels=1,
                                    n_classes=1)
          
   

In [ ]:
predict(predict_config) 

In [ ]:
plot_data_results(predict_config=predict_config, num_samples=3, fig_size=(30,20))

In [ ]:
import tifffile as tiff
from skimage.transform import resize
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt
import random
import numpy as np
from glob import glob
import os
H=10
images = [os.path.basename(f) for f in sorted(glob(os.path.join(predict_config.results_path, '*.tif')))]

# Sample
#images = images[:H] 
images = random.sample(images, H)
for img_file in images:
    try:
       #s2 3D
        img_s2 = tiff.imread(os.path.join(predict_config.images_path, 'images', img_file))
        img_s2 = minmax_scale(img_s2.ravel(), feature_range=(0, 255)).reshape(img_s2.shape)
        img_s2 = resize(img_s2, (predict_config.height, predict_config.width), mode='constant', preserve_range=True).astype(np.uint8)
       
        print(img_file)
        
        # Prediccion
        mask_ = tiff.imread(os.path.join(predict_config.results_path, img_file)) / 255
        mask_ = resize(mask_, (predict_config.height, predict_config.width), mode='constant', preserve_range=True)    

        fig, axes = plt.subplots(nrows=1, ncols=2 , figsize=(20, 40)) 
        
       
        axes[0].imshow(img_s2)
        axes[1].imshow(np.squeeze(mask_))
        
        
        plt.show()
        print("=================================================================================")
    except Exception as err:
        print(err)